In [1]:
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
hc_files = Path('./data/hra/control').glob('*.csv')
ad_files = Path('./data/hra/dementia').glob('*.csv')

lower_list = [chr(x) for x in range(97, 123)]
digit2id_list = list(range(10,33))

hc = []
for f in hc_files:
    df = pd.read_csv(f)
    s = df['x']
    
    for i in range(len(digit2id_list)):
        s = s.replace(digit2id_list[i], lower_list[i])
        
    hc.append(''.join([str(c) for c in s.to_list()]))
    
ad = []
for f in ad_files:
    df = pd.read_csv(f)
    s = df['x']
    
    for i in range(len(digit2id_list)):
        s = s.replace(digit2id_list[i], lower_list[i])
        
    ad.append(''.join([str(i) for i in s.to_list()]))

In [33]:
char_vectorizer_hc = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1,1),
    sublinear_tf=True)

char_vectorizer_ad = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1,1),
    sublinear_tf=True)

tfidf_hc = char_vectorizer_hc.fit_transform(hc)
tfidf_ad = char_vectorizer_ad.fit_transform(ad)

tfidf_mx_hc = pd.DataFrame(data=tfidf_hc.toarray(), columns=char_vectorizer_hc.get_feature_names())
tfidf_mx_ad = pd.DataFrame(data=tfidf_ad.toarray(), columns=char_vectorizer_ad.get_feature_names())
label1 = [0 for _ in range(tfidf_mx_hc.shape[0])]
label2 = [1 for _ in range(tfidf_mx_ad.shape[0])]

In [34]:
tfidf_mx = pd.concat([tfidf_mx_hc,tfidf_mx_ad])
rank_mx = tfidf_mx.rank(axis=1,method='first',ascending=False)
rank_mx['label'] = label1+label2
rank_mx

,0,1,2,3,4,5,6,7,8,9,...,n,o,p,q,r,s,t,u,v,label
0,1.0,29.0,30.0,21.0,13.0,31.0,11.0,22.0,18.0,15.0,...,9.0,5.0,4.0,16.0,27.0,20.0,19.0,23.0,2.0,0
1,1.0,27.0,28.0,18.0,11.0,29.0,10.0,19.0,15.0,12.0,...,9.0,6.0,4.0,20.0,26.0,22.0,14.0,23.0,32.0,0
2,1.0,21.0,23.0,14.0,12.0,28.0,9.0,24.0,18.0,13.0,...,11.0,6.0,3.0,19.0,30.0,22.0,31.0,20.0,32.0,0
3,1.0,12.0,29.0,28.0,15.0,25.0,7.0,26.0,18.0,14.0,...,11.0,9.0,2.0,16.0,24.0,20.0,31.0,17.0,32.0,0
4,1.0,27.0,29.0,26.0,10.0,18.0,3.0,21.0,15.0,13.0,...,9.0,7.0,2.0,17.0,24.0,20.0,31.0,22.0,32.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1.0,27.0,29.0,17.0,12.0,24.0,10.0,22.0,19.0,14.0,...,11.0,7.0,3.0,18.0,28.0,16.0,31.0,26.0,32.0,1
305,1.0,26.0,29.0,17.0,14.0,30.0,9.0,22.0,21.0,10.0,...,12.0,8.0,2.0,15.0,27.0,20.0,31.0,23.0,32.0,1
306,1.0,28.0,29.0,18.0,16.0,12.0,9.0,21.0,20.0,10.0,...,11.0,7.0,2.0,15.0,25.0,23.0,31.0,24.0,32.0,1
307,1.0,26.0,27.0,12.0,10.0,28.0,6.0,24.0,16.0,18.0,...,11.0,7.0,2.0,21.0,20.0,25.0,31.0,22.0,32.0,1


In [36]:
rank_mx.to_csv('tf-df-rank_mx.csv',index=False)